# Getting Started with next.jdbc

この`next.jdbc`ライブラリは、[`clojure.java.jdbc`](https://github.com/clojure/java.jdbc)Contribライブラリに代わるよりシンプルで高速な代替手段を提供し、そのライブラリの進化の次のステップです。

Clojure 1.10以降で動作するように設計されており、 `datafy`/`nav`をサポートし、データベースがサポートしている場合、デフォルトでソーステーブルと列名（ラベル）を示す、自動的に修飾されたキーワードでハッシュマップを生成します。

## Installation

次のいずれかを使用して、あなたのプロジェクトに`next.jdbc`を追加できます。

`deps.edn`向けには

```clojure
{seancorfield/next.jdbc {:mvn/version "1.0.10"}}
```

`project.clj`または`build.boot`向けには

```clojure
[seancorfield/next.jdbc "1.0.10"]
```

さらに、使用しているデータベースに使用するJDBCドライバーの依存を追加する必要があります。 テストされた`next.jdbc`のドライバーとバージョンは[プロジェクトの`deps.edn`ファイル](https://github.com/seancorfield/next-jdbc/blob/master/deps.edn#L11-L24)で確認できますが、他のデータベース用の他の多くのJDBCドライバーも動作するはずです（例：Oracle、Red Shift）。

## An Example REPL Session

`next.jdbc`の使用を開始するには、データソース（`javax.sql.DataSource`のインスタンス）を作成する必要があります。
`next-jdbc/get-datasource`は、「db-spec」（接続するデータベースを説明するハッシュマップ）、またはJDBC URI文字列とともに使用できます。
または、[HikariCP](https://brettwooldridge.github.io/HikariCP/)や[c3p0](https://www.mchange.com/projects/c3p0/)など、接続プーリングライブラリの1つからデータソースを構築できます。 -- 以下の[接続プーリング](＃connection-pooling)を参照してください。

このドキュメントの例では、ディスク上のローカルH2データベースを使用し、[Clojure CLI tools](https://clojure.org/guides/deps_and_cli)と `deps.edn`を使用します。

```clojure
;; deps.edn
{:deps {org.clojure/clojure {:mvn/version "1.10.1"}
        seancorfield/next.jdbc {:mvn/version "1.0.10"}
        com.h2database/h2 {:mvn/version "1.4.199"}}}
```


### Create & Populate a Database

このREPLセッションでは、H2データソースを定義し、単純なテーブルを使用してデータベースを作成し、データを追加してクエリを実行します。


```clojure
> clj
Clojure 1.10.1
user=> (require '[next.jdbc :as jdbc])
nil
user=> (def db {:dbtype "h2" :dbname "example"})
#'user/db
user=> (def ds (jdbc/get-datasource db))
#'user/ds
user=> (jdbc/execute! ds ["
create table address (
  id int auto_increment primary key,
  name varchar(32),
  email varchar(255)
)"])
[#:next.jdbc{:update-count 0}]
user=> (jdbc/execute! ds ["
insert into address(name,email)
  values('Sean Corfield','sean@corfield.org')"])
[#:next.jdbc{:update-count 1}]
user=> (jdbc/execute! ds ["select * from address"])
[#:ADDRESS{:ID 1, :NAME "Sean Corfield", :EMAIL "sean@corfield.org"}]
user=>
```


### The "db-spec" hash map

データベースはローカルファイルとして作成され、認証を必要としないため、データベースを `:dbtype`と`:dbname`だけで記述しました。
ほとんどのデータベースでは、認証に `:user`と`:password`が必要です。
データベースがリモートマシンで実行されている場合は、 `:host`と、場合によっては`:port`が必要になります（ `next.jdbc`は`:dbtype`に基づいて正しいポートを推測します）。

> 注：[next.jdbc/get-datasource](https://cljdoc.org/d/seancorfield/next.jdbc/CURRENT/api/next.jdbc#get-datasource) のdocstringでサポートされている `:dbtype`値の完全なリストを見ることができます。プログラムでこれが必要な場合は、[next.jdbc.connection/dbtypes](https://cljdoc.org/d/seancorfield/next.jdbc/CURRENT/api/next.jdbc.connection#dbtypes) ハッシュマップから取得できます。これらのリストが異なる場合、ハッシュマップが最終的なリストになります（docstringを修正する必要があります！）。そのVarのdocstringは、追加データベースについて `next.jdbc`に伝える方法を説明しています。


すでにJDBC URL（文字列）がある場合は、db-specハッシュマップの代わりにそのまま使用できます。 JDBC URLがあり、JDBCドライバーに追加のオプションを渡す必要がある場合は、文字列と必要な追加オプションを指定する `:jdbcUrl`キーでハッシュマップを使用できます。

### `execute!` & `execute-one!`

`execute!`を使用して `address`テーブルを作成し、新しい行を挿入し、クエリを実行しました。
3つの場合すべてで、 `execute!`は、操作からの結果セットを表すネームスペース修飾キーを持つハッシュマップのベクトルを返します（使用可能な場合）。
結果セットに行が含まれていない場合、`execute!`は空のベクター`[]`を返します。
結果セットが生成されない場合、 `next.jdbc`は操作からの「更新カウント」を含む「結果セット」を返します（通常は影響を受ける行の数です。`:builder-fn`はこの偽物の「結果セット」に影響しないことに注意してください。）。
デフォルトでは、H2は大文字の名前を使用し、 `next.jdbc`はこれらをそのまま返します。

単一の行（結果セットの最初の行、生成されたキー、または更新カウント）のみが必要な場合は、代わりに`execute-one!`を使用できます。 REPLセッションを継続して、別のアドレスを挿入し、生成されたキーを返すように要求してから、単一の行を照会します。

```clojure
user=> (jdbc/execute-one! ds ["
insert into address(name,email)
  values('Someone Else','some@elsewhere.com')
"] {:return-keys true})
#:ADDRESS{:ID 2}
user=> (jdbc/execute-one! ds ["select * from address where id = ?" 2])
#:ADDRESS{:ID 2, :NAME "Someone Else", :EMAIL "some@elsewhere.com"}
user=>
```

`execute-one!`を使用したため、1行だけが返されます（ハッシュマップ）。
これは、SQLステートメントにパラメーターを提供する方法も示しています -- SQLで`?`を使用し、SQL文字列の後のベクターで対応するパラメーター値を使用します。
結果セットに行が含まれていない場合、`execute-one!`は`nil`を返します。
結果が生成されず、`next.jdbc`が「更新カウント」を含む偽の「結果セット」を返す場合、`execute-one!`はキー`next.jdbc/update-count`を持つ単一のハッシュマップと更新された行数を返します。

> 注：一般に、更新カウントのみを取得するため、DDL操作には`execute-one!`を使用する必要があります。更新カウントのみを返すことがわかっているSQLステートメントがある場合は、`execute-one!`が正しい選択です。結果セットの単一の行のみを返すことがわかっているSQLステートメントがある場合は、おそらく`execute-one!`を使用する必要があります。結果セットで複数の行を返すSQLステートメントに`execute-one!`を使用する場合、最初の行のみを（ハッシュマップとして）取得しますが、結果セット全体は引き続きデータベースから取得されます。-- それはSQLを制限しません。

### Options & Result Set Builders

`next.jdbc`のすべての関数（`get-datasource`を除く）は、オプションの最後の引数として、 `next.jdbc`関数の動作を制御する[いくつかのオプション](/doc/all-the-options.md)を含むハッシュマップを受けることができる。

上記の `execute-one!`関数のオプションとして `:return-keys`が提供され、そのすぐ上で`:builder-fn`オプションに言及しました。
前述のように、デフォルトの動作では、テーブル名を修飾子としてカラム名を識別する名前空間修飾キーワードを含むハッシュマップとしてrowを返します。
[結果セットビルダー]（/doc/result-set-builders.md）に関する章全体がありますが、代わりに、修飾されていない小文字のキーワードを取得する方法を示す簡単な例を示します。

```clojure
user=> (require '[next.jdbc.result-set :as rs])
nil
user=> (jdbc/execute-one! ds ["
insert into address(name,email)
  values('Someone Else','some@elsewhere.com')
"] {:return-keys true :builder-fn rs/as-unqualified-lower-maps})
{:id 2}
user=> (jdbc/execute-one! ds ["select * from address where id = ?" 2]
                          {:builder-fn rs/as-unqualified-lower-maps})
{:id 2, :name "Someone Else", :email "some@elsewhere.com"}
user=>
```

デフォルトの結果セットビルダー（およびテーブル修飾されたカラム名）に依存することは、可能であれば、いくつかの注意を払って推奨されるアプローチです。
* デフォルトでは、MS SQL Serverは非修飾カラム名を生成します（MS SQL Serverからテーブル名を取得する方法については、[** Tips＆Tricks **](/doc/getting-started/friendly-sql-functions.md#tips--tricks) を参照してください)
* OracleのJDBCドライバーは `.getTableName()`をサポートしていないため、修飾されていないカラム名のみを生成します（**Tips & Tricks**でも説明されています）
* SQLクエリが重複したカラム名を生成する方法でテーブルを結合し、修飾されていないカラム名を使用する場合、重複したカラム名は競合し、結果でそのうちの1つのみを取得します。 -- SQLの（`as`）エイリアスを使用してカラム名を明確にしてください。
* SQLクエリが異なるエイリアスの下でテーブルをそれ自体に結合する場合、_qualified_カラム名は、クエリで使用したエイリアスではなく、JDBCドライバによって提供される基になるテーブル名に基づいているため競合します。-- 再び、SQLでエイリアスを使用してこれらのカラム名を明確にします。

### `plan` & Reducing Result Sets


### `plan` & Reducing Result Sets

While those functions are fine for retrieving result sets as data, most of the time you want to process that data efficiently, so `next.jdbc` provides a SQL execution function that works with `reduce` and with transducers to consume the result set without the intermediate overhead of creating Clojure data structures for every row:

```clojure
user=> (into #{}
             (map :ADDRESS/NAME)
             (jdbc/plan ds ["select * from address"]))
#{"Sean Corfield" "Someone Else"}
user=>
```

This produces a set of all the unique names in the `address` table, directly from the `java.sql.ResultSet` object returned by the JDBC driver, without creating any Clojure hash maps. That means you can use either the qualified keyword that would be produced by `execute!` or `execute-one!` or you can use a simple keyword that mirrors the column name (label) directly:

```clojure
user=> (into #{}
             (map :name)
             (jdbc/plan ds ["select * from address"]))
#{"Sean Corfield" "Someone Else"}
user=>
```

Any operation that can perform key-based lookup can be used here without creating hash maps: `get`, `contains?`, `find` (returns a `MapEntry` of whatever key you requested and the corresponding column value), or direct keyword access as shown above. Any operation that would require a Clojure hash map, such as `assoc` or anything that invokes `seq` (`keys`, `vals`), will cause the full row to be expanded into a hash map, such as produced by `execute!` or `execute-one!`.

> Note: since `plan` expects you to process the result set via reduction, you should not use it for DDL or for SQL statements that only produce update counts.

## Datasources, Connections & Transactions

In the examples above, we created a datasource and then passed it into each function call. When `next.jdbc` is given a datasource, it creates a `java.sql.Connection` from it, uses it for the SQL operation (by creating and populating a `java.sql.PreparedStatement` from the connection and the SQL string and parameters passed in), and then closes it. If you're not using a connection pooling datasource (see below), that can be quite an overhead: setting up database connections to remote servers is not cheap!

If you want to run multiple SQL operations without that overhead each time, you can create the connection yourself and reuse it across several operations using `with-open` and `next.jdbc/get-connection`:

```clojure
(with-open [con (jdbc/get-connection ds)]
  (jdbc/execute! con ...)
  (jdbc/execute! con ...)
  (into [] (map :column) (jdbc/plan con ...)))
```

If any of these operations throws an exception, the connection will still be closed but operations prior to the exception will have already been committed to the database. If you want to reuse a connection across multiple operations but have them all rollback if an exception occurs, you can use `next.jdbc/with-transaction`:

```clojure
(jdbc/with-transaction [tx ds]
  (jdbc/execute! tx ...)
  (jdbc/execute! tx ...)
  (into [] (map :column) (jdbc/plan tx ...)))
```

If `with-transaction` is given a datasource, it will create and close the connection for you. If you pass in an existing connection, `with-transaction` will set up a transaction on that connection and, after either committing or rolling back the transaction, will restore the state of the connection and leave it open:

```clojure
(with-open [con (jdbc/get-connection ds)]
  (jdbc/execute! con ...) ; committed
  (jdbc/with-transaction [tx con] ; will commit or rollback this group:
    (jdbc/execute! tx ...)
    (jdbc/execute! tx ...)
    (into [] (map :column) (jdbc/plan tx ...)))
  (jdbc/execute! con ...)) ; committed
```

You can read more about [working with transactions](/doc/transactions.md) further on in the documentation.

### Prepared Statement Caveat

Not all databases support using a `PreparedStatement` for every type of SQL operation. You might have to create a `java.sql.Statement` yourself, directly from a `java.sql.Connection` and use that, without parameters, in `plan`, `execute!`, or `execute-one!`. See the following example:

```clojure
(with-open [con (jdbc/get-connection ds)]
  (jdbc/execute! (.createStatement con) ["...just a SQL string..."])
  (jdbc/execute! con ["...some SQL..." "and" "parameters"]) ; uses PreparedStatement
  (into [] (map :column) (jdbc/plan (.createStatement con) ["..."])))
```

## Connection Pooling

`next.jdbc` makes it easy to use either HikariCP or c3p0 for connection pooling.

First, you need to add the connection pooling library as a dependency, e.g.,

```clojure
com.zaxxer/HikariCP {:mvn/version "3.3.1"}
;; or:
com.mchange/c3p0 {:mvn/version "0.9.5.4"}
```

_Check those libraries' documentation for the latest version to use!_

Then import the appropriate classes into your code:

```clojure
(ns my.main
  (:require [next.jdbc :as jdbc]
            [next.jdbc.connection :as connection])
  (:import (com.zaxxer.hikari HikariDataSource)
           ;; or:
           (com.mchange.v2.c3p0 ComboPooledDataSource PooledDataSource)))
```

Finally, create the connection pooled datasource. `db-spec` here contains the regular `next.jdbc` options (`:dbtype`, `:dbname`, and maybe `:host`, `:port`, `:classname` etc -- or the `:jdbcUrl` format mentioned above). Those are used to construct the JDBC URL that is passed into the datasource object (by calling `.setJdbcUrl` on it). You can also specify any of the connection pooling library's options, as mixed case keywords corresponding to any simple setter methods on the class being passed in, e.g., `:connectionTestQuery`, `:maximumPoolSize` (HikariCP), `:maxPoolSize`, `:preferredTestQuery` (c3p0).

Some important notes regarding HikariCP:

* Authentication credentials must use `:username` (if you are using c3p0 or regular, non-pooled, connections, then the db-spec hash map must contain `:user`).
* When using `:dbtype "jtds"`, you must specify `:connectionTestQuery "SELECT 1"` (or some other query to verify the health of a connection) because the jTDS JDBC driver does not implement `.isValid()` so HikariCP requires a specific test query instead (c3p0 does not rely on this method so it works with jTDS without needing `:preferredTestQuery`).

You will generally want to create the connection pooled datasource at the start of your program (and close it before you exit, although that's not really important since it'll be cleaned up when the JVM shuts down):

```clojure
(defn -main [& args]
  (with-open [^HikariDataSource ds (connection/->pool HikariDataSource db-spec)]
    (jdbc/execute! ds ...)
    (jdbc/execute! ds ...)
    (do-other-stuff ds args)
    (into [] (map :column) (jdbc/plan ds ...))))
;; or:
(defn -main [& args]
  (with-open [^PooledDataSource ds (connection/->pool ComboPooledDataSource db-spec)]
    (jdbc/execute! ds ...)
    (jdbc/execute! ds ...)
    (do-other-stuff ds args)
    (into [] (map :column) (jdbc/plan ds ...))))
```

You only need the type hints on `ds` if you plan to call methods on it via Java interop, such as `.close` (or using `with-open` to auto-close it) and you want to avoid reflection.

If you are using [Component](https://github.com/stuartsierra/component), a connection pooled datasource is a good candidate since it has a `start`/`stop` lifecycle:

```clojure
(ns ...
  (:require [com.stuartsierra.component :as component]
            ...))

(defrecord Database [db-spec ^HikariDataSource datasource]
  component/Lifecycle
  (start [this]
    (if datasource
      this ; already started
      (assoc this :datasource (connection/->pool HikariDataSource db-spec))))
  (stop [this]
    (if datasource
      (do
        (.close datasource)
        (assoc this :datasource nil))
      this))) ; already stopped

(defn -main [& args]
  (let [db (component/start (map->Database {:db-spec db-spec}))]
    (try
      (jdbc/execute! (:datasource db) ...)
      (jdbc/execute! (:datasource db) ...)
      (do-other-stuff db args)
      (into [] (map :column) (jdbc/plan (:datasource db) ...)))
      (catch Throwable t)
        (component/stop db)))
```

## Working with Additional Data Types

By default, `next.jdbc` relies on the JDBC driver to handle all data type conversions when reading from a result set (to produce Clojure values from SQL values) or setting parameters (to produce SQL values from Clojure values). Sometimes that means that you will get back a database-specific Java object that would need to be manually converted to a Clojure data structure, or that certain database column types require you to manually construct the appropriate database-specific Java object to pass into a SQL operation. You can usually automate those conversions using either the [ReadableColumn protocol](/doc/result-set-builders.md#readablecolumn) (for converting database-specific types to Clojure values) or the [SettableParameter protocol](/doc/prepared-statements.md#prepared-statement-parameters) (for converting Clojure values to database-specific types).

## Support from Specs

As you are developing with `next.jdbc`, it can be useful to have assistance from `clojure.spec` in checking calls to `next.jdbc`'s functions, to provide explicit argument checking and/or better error messages for some common mistakes, e.g., trying to pass a plain SQL string where a vector (containing a SQL string, and no parameters) is expected.

You can enable argument checking for functions in `next.jdbc`, `next.jdbc.connection`, `next.jdbc.prepare`, and `next.jdbc.sql` by requiring the `next.jdbc.specs` namespace and instrumenting the functions. A convenience function is provided:

```clojure
(require '[next.jdbc.specs :as specs])
(specs/instrument) ; instruments all next.jdbc API functions

(jdbc/execute! ds "SELECT * FROM fruit")
Call to #'next.jdbc/execute! did not conform to spec.
```

In the `:problems` output, you'll see the `:path [:sql :sql-params]` and `:pred vector?` for the `:val "SELECT * FROM fruit"`. Without the specs' assistance, this mistake would produce a more cryptic error, a `ClassCastException`, that a `Character` cannot be cast to a `String`, from inside `next.jdbc.prepare`.

A convenience function also exists to revert that instrumentation:

```clojure
(specs/unstrument) ; undoes the instrumentation of all next.jdbc API functions
```

[Friendly SQL Functions :>](/doc/friendly-sql-functions.md)